In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

# Column selection

We start by loading the raw test data.

In [2]:
raw_data = pd.read_csv('../data/test_raw.csv', sep=';').T
raw_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,15088,15089,15090,15091,15092,15093,15094,15095,15096,15097
row ID,8473.000000,5467.000000,14642.000000,14644.000000,14640.000000,14641.000000,14643.000000,13576.000000,11315.000000,9161.000000,...,5808.000000,5663.000000,7874.000000,4686.000000,4687.000000,13756.000000,8116.000000,3523.000000,5664.000000,9160.000000
row m/z,95.046412,96.078154,97.965959,97.966018,97.966026,97.966036,97.966038,98.093955,99.077666,99.077760,...,1188.647673,1189.649801,1190.652087,1193.744747,1194.748002,1194.930725,1195.751831,1196.600380,1197.603464,1199.638546
row retention time,3.561787,7.262401,2.058559,2.379738,1.716301,1.910083,2.138584,1.647483,11.761351,9.571574,...,5.122062,4.809316,4.777540,10.285306,10.287319,1.828250,10.316287,4.792202,4.790557,5.761941
NT_20180919_569_RB4_01_31949.mzXML Peak height,176.000000,57.000000,0.000000,0.000000,554.000000,317.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NT_20180919_569_RB4_01_31949.mzXML Peak area,753.501500,263.032000,0.000000,0.000000,2874.741000,1551.641000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NT_20180919_568_RB3_01_31948.mzXML Peak height,238.000000,27.000000,0.000000,0.000000,776.000000,455.000000,0.000000,0.000000,0.000000,113.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NT_20180919_568_RB3_01_31948.mzXML Peak area,818.162500,128.808000,0.000000,0.000000,3216.031000,2144.516000,0.000000,0.000000,0.000000,299.460000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NT_20180919_562_RA5_01_31942.mzXML Peak height,0.000000,142.000000,0.000000,0.000000,625.000000,495.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NT_20180919_562_RA5_01_31942.mzXML Peak area,0.000000,1598.882500,0.000000,0.000000,2879.395500,2168.672000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NT_20180919_563_RA6_01_31943.mzXML Peak height,0.000000,164.000000,0.000000,0.000000,724.000000,496.000000,0.000000,0.000000,129.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Now we obtain the metabolite properties

In [3]:
peak_IDs = raw_data.T[['row m/z', 'row retention time']].rename(index=str,
                       columns={'row m/z': 'm/z', 'row retention time': 'retention time'})
peak_IDs.head()

,m/z,retention time
0,95.046412,3.561787
1,96.078154,7.262401
2,97.965959,2.058559
3,97.966018,2.379738
4,97.966026,1.716301


Here we load the properties of the most important metabolites identified from the training set

In [4]:
peak_train = pd.read_csv('../data/peak_IDs.csv').loc[[16674,20216,38242]]
peak_train

,m/z,retention time,n peaks
16674,155.069762,7.568337,90.0
20216,368.140123,7.564014,85.0
38242,367.136409,7.566092,141.0


We need to find the closest match for these metabolites in the test set. We look for the closest mass; we impose the constraint, however, that the retention time also needs to be close to the original&mdash;we set the limit to 5%.

In [5]:
mass_train = peak_train['m/z'].values
time_train = peak_train['retention time'].values

idx_test = []

for mass, time in zip(mass_train, time_train):
    diff_time = np.abs(peak_IDs['retention time'].values - time)/peak_IDs['retention time'].values
    mask_times = (diff_time <= 0.05)
    peaks = peak_IDs.loc[mask_times]
    diff_mass = np.abs(peaks['m/z'].values - mass)
    idx = peaks.iloc[np.argmin(diff_mass)].name
    idx_test.append(int(idx))

In [6]:
peak_IDs.iloc[idx_test]

,m/z,retention time
614,155.068646,7.637162
6025,368.139917,7.614992
6010,367.136464,7.673215


We remove all columns from the test set, except for these three. We further remove the rows corresponding to blanks, and clean the sample IDs.

In [7]:
data = raw_data.drop(raw_data.index[[i for i in range(3)]])[idx_test]
data.head()

,614,6025,6010
NT_20180919_569_RB4_01_31949.mzXML Peak height,9324.0000,15211.0000,80281.0000
NT_20180919_569_RB4_01_31949.mzXML Peak area,63882.2390,73509.1155,388906.2375
NT_20180919_568_RB3_01_31948.mzXML Peak height,11600.0000,37736.0000,207161.0000
NT_20180919_568_RB3_01_31948.mzXML Peak area,61165.0155,131483.4495,731149.0935
NT_20180919_562_RA5_01_31942.mzXML Peak height,7272.0000,18185.0000,95338.0000


In [8]:
id_blanks = ['559','560','561','631','632','633','652','653','654','742','743','744']
mask_blanks = np.zeros(data.shape[0])
for blank in id_blanks:
    mask_blanks = mask_blanks + np.array([blank in x for x in data.index.values])

mask_blanks = mask_blanks.astype(bool)
idx_blanks = data[mask_blanks].index.values

In [9]:
data = data.drop(idx_blanks)
data.head()

,614,6025,6010
NT_20180919_569_RB4_01_31949.mzXML Peak height,9324.0000,15211.0000,80281.0000
NT_20180919_569_RB4_01_31949.mzXML Peak area,63882.2390,73509.1155,388906.2375
NT_20180919_568_RB3_01_31948.mzXML Peak height,11600.0000,37736.0000,207161.0000
NT_20180919_568_RB3_01_31948.mzXML Peak area,61165.0155,131483.4495,731149.0935
NT_20180919_562_RA5_01_31942.mzXML Peak height,7272.0000,18185.0000,95338.0000


In [10]:
y = [x[-x[::-1].find('_'):] for x in data.index]
data.index = y
data.head()

,614,6025,6010
31949.mzXML Peak height,9324.0000,15211.0000,80281.0000
31949.mzXML Peak area,63882.2390,73509.1155,388906.2375
31948.mzXML Peak height,11600.0000,37736.0000,207161.0000
31948.mzXML Peak area,61165.0155,131483.4495,731149.0935
31942.mzXML Peak height,7272.0000,18185.0000,95338.0000


In [11]:
data = data.sort_index()
data.head()

,614,6025,6010
31942.mzXML Peak area,28555.9370,59482.587,284899.721
31942.mzXML Peak height,7272.0000,18185.000,95338.000
31943.mzXML Peak area,42929.5155,72773.481,375126.465
31943.mzXML Peak height,7362.0000,19273.000,116210.000
31944.mzXML Peak area,38809.3895,69676.185,292490.766


With this clean, we separate the intensity and AUC datasets. Before this, we transform all peaks to log scale.

In [12]:
data = np.log1p(data)
data.head()

,614,6025,6010
31942.mzXML Peak area,10.259655,10.993456,12.559896
31942.mzXML Peak height,8.891924,9.808407,11.465194
31943.mzXML Peak area,10.667338,11.195121,12.835021
31943.mzXML Peak height,8.904223,9.866512,11.663163
31944.mzXML Peak area,10.566443,11.151628,12.586192


In [13]:
intensity = ['height' in x for x in data.index]
area = ['area' in x for x in data.index]

data_int = data[intensity]
data_auc = data[area]

new_index = [x[:x.find('.')] for x in data_int.index]
data_int.index = new_index
data_auc.index = new_index

In [14]:
data_int.head()

,614,6025,6010
31942,8.891924,9.808407,11.465194
31943,8.904223,9.866512,11.663163
31944,8.465689,9.618801,11.316314
31945,9.561490,9.410993,11.132455
31946,9.173261,9.042158,10.672345


In [15]:
data_auc.head()

,614,6025,6010
31942,10.259655,10.993456,12.559896
31943,10.667338,11.195121,12.835021
31944,10.566443,11.151628,12.586192
31945,11.269538,11.161079,12.913611
31946,10.196067,10.067033,12.426411


We also generate a zeroed AUC dataset.

In [16]:
data_zauc = np.heaviside(data_int,0)*data_auc

data_zauc.head()

,614,6025,6010
31942,10.259655,10.993456,12.559896
31943,10.667338,11.195121,12.835021
31944,10.566443,11.151628,12.586192
31945,11.269538,11.161079,12.913611
31946,10.196067,10.067033,12.426411


We note, however, that for these three compounds the AUC and zeroed AUC data are the same.

In [18]:
(data_zauc == data_auc).all()

614     True
6025    True
6010    True
dtype: bool

# Predictions

In [26]:
def test_pred(train_data, test_data, target):
    
    predictions = pd.DataFrame(index=test_data.index)
    
    params = {'objective': 'binary', 'metric': 'auc', 'random_state': 0}
    for col in train_data.columns:
    
        feat_train = train_data[col].values.reshape(-1,1)
        feat_test = test_data[col].values.reshape(-1,1)
    
        train_set = lgb.Dataset(feat_train, label=target)
    
        bst = lgb.train(params, train_set, num_boost_round=3, verbose_eval=False)
    
        probs = bst.predict(feat_test)
    
        predictions[col] = probs
        
    return predictions

## 1. Intensity

In [19]:
train_data = pd.read_csv('../data/model/data_int_NZ.csv', index_col=0, usecols=['Unnamed: 0', '16674', '20216', '38242'])
train_data.head()

,16674,20216,38242
19777,8.278682,8.450198,10.268998
19778,0.000000,3.496508,5.648974
19779,0.000000,2.197225,5.342334
19780,0.000000,3.367296,5.771441
19781,0.693147,4.905275,5.998937


In [20]:
le = LabelEncoder()

genus = pd.read_csv('../data/model/metadata.csv', index_col=0).genus

train_data = train_data.loc[genus.index]

genus = le.fit_transform(genus)

train_data['target'] = genus

train_data.head()

,16674,20216,38242,target
19777,8.278682,8.450198,10.268998,1
19789,11.380525,10.966127,12.645475,1
19817,0.000000,3.688879,0.000000,0
19818,0.000000,4.382027,5.379897,0
19819,0.000000,3.951244,5.236442,0


For each of the columns, we retrain the model once on all training samples and predict the genera of the testing samples. Before running the model, we shuffle the training data and rename the columns of the testing data so that both match.

In [22]:
train_data = train_data.sample(frac=1)
train_data.head()

,16674,20216,38242,target
20090,0.000000,0.000000,0.000000,0
20063,0.000000,3.091042,0.000000,0
20025,9.529376,9.289521,11.009324,1
20082,0.000000,3.367296,4.804021,0
19960,0.000000,0.000000,0.000000,0


In [23]:
target = train_data.target.values

train_data = train_data.drop(['target'], axis=1)

In [24]:
data_int = data_int.rename(columns={614: '16674', 6025: '20216', 6010: '38242'})
data_int = data_int.sample(frac=1)
data_int.head()

,16674,20216,38242
32188,4.727388,0.000000,7.640604
31967,10.006180,11.287918,13.010041
32099,10.700995,9.589872,11.356470
32013,0.000000,0.000000,5.888878
31962,8.074960,8.257126,9.789198


In [45]:
probs_int = test_pred(train_data, data_int, target)
probs_int = probs_int.sort_index()
probs_int.head()

,16674,20216,38242
31942,0.634166,0.634166,0.634166
31943,0.634166,0.634166,0.634166
31944,0.634166,0.634166,0.634166
31945,0.634166,0.634166,0.634166
31946,0.634166,0.634166,0.621609


In [46]:
probs_int.to_csv('../data/model/probs_int.csv')

In [47]:
classes_int = np.heaviside(probs_int - 0.5, 1).astype(int)
classes_int.head()

,16674,20216,38242
31942,1,1,1
31943,1,1,1
31944,1,1,1
31945,1,1,1
31946,1,1,1


In [48]:
classes_int.to_csv('../data/model/classes_int.csv')

## 2. AUC

In [50]:
train_data = pd.read_csv('../data/model/data_auc_NZ.csv', index_col=0, usecols=['Unnamed: 0', '16674', '20216', '38242'])
train_data.head()

,16674,20216,38242
19777,10.149639,10.222877,11.929564
19778,0.000000,5.500560,7.165139
19779,3.493655,5.864889,6.427755
19780,3.979644,0.000000,7.270472
19781,4.479159,5.592983,6.800649


In [51]:
le = LabelEncoder()

genus = pd.read_csv('../data/model/metadata.csv', index_col=0).genus

train_data = train_data.loc[genus.index]

genus = le.fit_transform(genus)

train_data['target'] = genus

train_data.head()

,16674,20216,38242,target
19777,10.149639,10.222877,11.929564,1
19789,13.082517,12.437366,14.154770,1
19817,0.000000,6.250394,6.301011,0
19818,5.518091,5.024979,6.779354,0
19819,0.000000,2.496341,5.994308,0


For each of the columns, we retrain the model once on all training samples and predict the genera of the testing samples. Before running the model, we shuffle the training data and rename the columns of the testing data so that both match.

In [52]:
train_data = train_data.sample(frac=1)
train_data.head()

,16674,20216,38242,target
19919,0.000000,0.000000,0.000000,0
20096,0.000000,0.000000,0.000000,0
20165,0.000000,0.000000,3.638349,0
20030,8.558394,8.108230,8.304759,1
20091,9.542397,9.322775,10.665652,1


In [53]:
target = train_data.target.values

train_data = train_data.drop(['target'], axis=1)

In [54]:
data_auc = data_auc.rename(columns={614: '16674', 6025: '20216', 6010: '38242'})
data_auc = data_auc.sample(frac=1)
data_auc.head()

,16674,20216,38242
32086,0.000000,0.000000,0.000000
32008,7.445704,0.000000,8.102721
32177,6.942263,0.000000,7.601080
31965,11.325935,11.073615,12.750358
31975,9.513979,9.776515,11.540562


In [55]:
probs_auc = test_pred(train_data, data_auc, target)
probs_auc = probs_auc.sort_index()
probs_auc.head()

,16674,20216,38242
31942,0.634166,0.634166,0.634166
31943,0.634166,0.634166,0.634166
31944,0.634166,0.634166,0.634166
31945,0.634166,0.634166,0.634166
31946,0.634166,0.634166,0.634166


In [56]:
probs_auc.to_csv('../data/model/probs_auc.csv')

In [57]:
classes_auc = np.heaviside(probs_auc - 0.5, 1).astype(int)
classes_auc.head()

,16674,20216,38242
31942,1,1,1
31943,1,1,1
31944,1,1,1
31945,1,1,1
31946,1,1,1


In [58]:
classes_auc.to_csv('../data/model/classes_auc.csv')

## 3. Zeroed AUC

In [59]:
train_data = pd.read_csv('../data/model/data_zauc_NZ.csv', index_col=0, usecols=['Unnamed: 0', '16674', '20216', '38242'])
train_data.head()

,16674,20216,38242
19777,10.149639,10.222877,11.929564
19778,0.000000,5.500560,7.165139
19779,0.000000,5.864889,6.427755
19780,0.000000,0.000000,7.270472
19781,4.479159,5.592983,6.800649


In [60]:
le = LabelEncoder()

genus = pd.read_csv('../data/model/metadata.csv', index_col=0).genus

train_data = train_data.loc[genus.index]

genus = le.fit_transform(genus)

train_data['target'] = genus

train_data.head()

,16674,20216,38242,target
19777,10.149639,10.222877,11.929564,1
19789,13.082517,12.437366,14.154770,1
19817,0.000000,6.250394,0.000000,0
19818,0.000000,5.024979,6.779354,0
19819,0.000000,2.496341,5.994308,0


For each of the columns, we retrain the model once on all training samples and predict the genera of the testing samples. Before running the model, we shuffle the training data and rename the columns of the testing data so that both match.

In [61]:
train_data = train_data.sample(frac=1)
train_data.head()

,16674,20216,38242,target
20134,10.338479,10.054602,10.915558,1
20047,7.846613,7.971868,8.785481,1
19952,11.133668,11.456090,13.298468,1
20109,0.000000,0.000000,0.000000,0
19960,0.000000,0.000000,0.000000,0


In [62]:
target = train_data.target.values

train_data = train_data.drop(['target'], axis=1)

In [63]:
data_auc = data_auc.rename(columns={614: '16674', 6025: '20216', 6010: '38242'})
data_auc = data_auc.sample(frac=1)
data_auc.head()

,16674,20216,38242
32155,6.605061,0.000000,8.288612
31976,9.895768,10.135243,11.251994
32135,11.425848,10.816446,12.354180
32150,0.000000,6.861219,9.107305
31990,7.005083,6.614392,9.078992


In [64]:
probs_zauc = test_pred(train_data, data_auc, target)
probs_zauc = probs_zauc.sort_index()
probs_zauc.head()

,16674,20216,38242
31942,0.634166,0.634166,0.634166
31943,0.634166,0.634166,0.634166
31944,0.634166,0.634166,0.634166
31945,0.634166,0.634166,0.634166
31946,0.634166,0.634166,0.634166


In [65]:
probs_zauc.to_csv('../data/model/probs_zauc.csv')

In [66]:
classes_zauc = np.heaviside(probs_zauc - 0.5, 1).astype(int)
classes_zauc.head()

,16674,20216,38242
31942,1,1,1
31943,1,1,1
31944,1,1,1
31945,1,1,1
31946,1,1,1


In [67]:
classes_zauc.to_csv('../data/model/classes_zauc.csv')